# Problem Solving

In [11]:
import numojo as nm
from python import Python
from tensor import Tensor, TensorShape

## What is an algorithm?

In [2]:
var numbers = SIMD[DType.int32, 8](31, 4, 15, 92, 6, 5, 3, 5)
var max_value = numbers.reduce_max()
print(max_value)


92


In [3]:
print(numbers[6])

3


## Finding the Largest value in an arbitrary list

In [4]:
fn flawed[n: Int](A: SIMD[DType.int32, n]) -> SIMD[DType.int32, 1]:
    var my_max: SIMD[DType.int32, 1] = 0
    for v in range(len(A)):
        if A[v] > my_max:
            my_max = A[v]
    return my_max


In [5]:
max_value = flawed(numbers)
print(max_value)

92


In [6]:
## Corrected function
fn largest[n: Int](A: SIMD[DType.int32, n]) raises -> SIMD[DType.int32, 1]:
    var my_max: SIMD[DType.int32, 1] = A[0]
    for v in range(1, len(A)):
        if A[v] > my_max:
            my_max = A[v]
    return my_max

In [7]:
max_value = largest(numbers)
print(max_value)

92


## Models can predict algorithm performance

In [8]:
fn alternate[n: Int](A: SIMD[DType.int32, n]) -> SIMD[DType.int32, 1]:
    for v in range(len(A)):
        var v_is_largest = True
        for x in range(len(A)):
            if A[x] < A[v]:
                v_is_largest = False
                break
        if v_is_largest:
            return A[v]
    return 0

In [9]:
max_value = alternate[8](numbers)
print(max_value)
print(numbers)

3
[31, 4, 15, 92, 6, 5, 3, 5]


## Finding two largest values in an arbitrary list

In [10]:
fn largest_two[dtype: DType, size: Int](A: SIMD[dtype, size]) raises -> SIMD[dtype, 2]:
    
    var first: SIMD[dtype, 1] = A[0]
    var second: SIMD[dtype, 1] = A[1]

    if second > first:
        first, second = second, first

    for v in range(2, len(A)):
        if A[v] > first:
            second = first
            first = A[v]
        elif A[v] > second:
            second = A[v]

    return SIMD[dtype, 2](first, second)

fn largest_two[dtype: DType](A: nm.NDArray[dtype]) raises -> SIMD[dtype, 2]:
    
    var first: SIMD[dtype, 1] = A[0]
    var second: SIMD[dtype, 1] = A[1]

    if second > first:
        first, second = second, first

    for v in range(2, len(A)):
        if A[v] > first:
            second = first
            first = A[v]
        elif A[v] > second:
            second = A[v]

    return SIMD[dtype, 2](first, second)

In [11]:
var two_max = largest_two(numbers)
print(two_max)
print(numbers)


[92, 31]
[31, 4, 15, 92, 6, 5, 3, 5]


In [74]:
fn tournament_two(A: PythonObject) raises -> (Int, Int):

    var N = len(A)
    var winner = PythonObject([])
    var loser = PythonObject([])
    var prior = PythonObject([])
    var idx = 0
    var m = 0

    for i in range(N - 1):
        winner.append(Python.none())    # 1
        loser.append(Python.none())
        prior.append(-1)                # 2
    
    for i in range(0, N, 2):            # 3
        if A[i] < A[i + 1]:
            winner[idx] = A[i + 1]
            loser[idx] = A[i]
        else:
            winner[idx] = A[i]
            loser[idx] = A[i + 1]
        idx += 1
    
    while idx < (N - 1):                # 4
        if winner[m] < winner[m + 1]:   # 5
            winner[idx] = winner[m + 1]
            loser[idx] = winner[m]
            prior[idx] = m + 1
        else:
            winner[idx] = winner[m]
            loser[idx] = winner[m + 1]
            prior[idx] = m
        m += 2                          # 6
        idx += 1
    
    var largest = winner[m]
    var second = loser[m]              # 7  
    m = prior[m]

    while m >= 0:
        if second < loser[m]:           # 8
            second = loser[m]
        m = prior[m]
    
    return (Int(largest), Int(second))

In [75]:
var numbers = PythonObject([31, 4, 15, 92, 6, 5, 3, 5])
var winners = tournament_two(numbers)
print("The winner is", winners[0], "and the runner up is", winners[1])

The winner is 92 and the runner up is 31
